In [1]:
from PIL import Image
import pytesseract
from pytesseract import Output
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import re

#### To make your own ticker array. 

1. Change the name and save the file to load it later.

#### Creating a folder structure for all the screenshots.

1. Rohit_data   >>>   Companies   >>>   All tickers   >>>   In each ticker two folders with suppliers and customers. 

In [12]:
supplier_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Cost',
           'Cost Cat', '% Supplier Rev', 'Source', 'Size Source']
customer_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Rev',
           '% Customer Cost', 'Customer Cost Cat', 'Source', 'Size Source']

# Read files in folder

In [34]:
import os
rootdir = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VWR/Customers'
def listdirs(rootdir):
    file_list = []
    for file in os.listdir(rootdir):
        print(rootdir + "/"+file)
        file_list.append(rootdir + "/"+file)
    return file_list
        

In [35]:
file_list = listdirs(rootdir)

In [36]:
file_list

[]

In [4]:
# Define the replacement function
def replace_em_dash(cell_value):
    if isinstance(cell_value, str) and '—' in str(cell_value):
        return '--'
    else:
        return cell_value

# Define the replacement function
def replace_no_numbers(cell_value):
    if pd.notna(cell_value) and not bool(re.search(r'\d', str(cell_value))):
        return '--'
    return cell_value

In [5]:
def customer_file(data):
    df = pd.DataFrame(data)
    df = df[['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Rev',
           '% Customer Cost', 'Customer Cost Cat', 'Source']]
    df["Source"] = df["Source"].str.extract(r'(\d{4})')
    name = df['Company'][0]
    df = df.rename(columns = {"Company":"Customer Name"})
    df.insert(0, "Firm Name", name)
    df = df.iloc[1:]
    
    # replace em dash with "--"
    df = df.applymap(replace_em_dash)
    # Mk Cap, Relationship Size, % Rev, % Customer Cost has to have numerical values if not "--"
    df[['Mkt Cap (M)', 'Relationship Size (M)', '% Rev','% Customer Cost']] = df[['Mkt Cap (M)', 
                                                                                  'Relationship Size (M)', 
                                                                                  '% Rev',
                                                                                  '% Customer Cost']].applymap(replace_no_numbers)

#     customer_dict = df.set_index('Company').agg(list, axis=1).to_dict()
#     arr = np.array([[ticker,name,customer_dict]])
    return df

## Load ticker files

In [6]:
tickers = np.load('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/thao_tickers.npy', allow_pickle=True)

In [7]:
tickers = tickers[(tickers != "DSKE")&(tickers != "MCFT")]

In [37]:
for ticker in tickers:
    rootdir = "/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/" + ticker + "/Customers"
    file_list = listdirs(rootdir)
    if file_list == []:
        continue    
    df = pd.DataFrame()
    for path in file_list:
        data = pd.read_csv(path)
#         df.append(supplier_file(data))
        df = pd.concat([df, customer_file(data)], axis=0)
    
    # long name will end with "..."
    dot_pattern = r'(\.|\.\.|\.\.\.|\s\.|\s\.\.|\s\.\.\.)$'
    df['Customer Name'] = df['Customer Name'].str.replace(dot_pattern, '...', regex=True)
    df['Industry'] = df['Industry'].str.replace(dot_pattern, '...', regex=True)
    # treat long name of Firm Name
    if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
        df['Firm Name'] = df['Firm Name'].str.replace(dot_pattern, '...', regex=True)
        not_matching_pattern = ~df['Firm Name'].str.contains(dot_pattern)
        df.loc[not_matching_pattern, 'Firm Name'] = df.loc[not_matching_pattern, 'Firm Name'] + '...'    
    df = df.drop_duplicates(subset=['Customer Name', 'Industry','Source']).reset_index(drop=True)
    df.insert(0, "ticker", ticker)
    df.to_csv("/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/" + ticker +"/customers.csv",index = False)        
    

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/FRPT/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/SKYS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BSGM/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ANN/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VIVE/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CHRS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/TLCC/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MRMD/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MTRAF/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/FGEN/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/MEVI/Customers/MEVI C 

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regul

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HABT/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CRBP/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/SIEN/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/DIIBF/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXTA/Customers/C 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXTA/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/UNIR/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/UFAB/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/KSWS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BLCM/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/XLNX/Customers/C 2.cs

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  not_matching_pattern = ~df['Firm Name'].str.contains(dot_pattern)
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular exp

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NRXCF/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers/C 3.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers/C 2.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/TLRY/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/RSTRF/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/PARR/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ACB/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CRON/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/PIRS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/LCINQ/Customers/C 1.c

...

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regul

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/CTMX/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NVCR/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HOME/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/RGNX/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/PEN/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/NBRV/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Customers/C 18.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Customers/C 19.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Customers/C 9.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Customers/C 8.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/HPE/Customers/C 20.csv
/U

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:19: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  not_matching_pattern = ~df['Firm Name'].str.contains(dot_pattern)
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular exp

/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/ANAB/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/IOTS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/WAAS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/DLTH/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/XCOMQ/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/WVE/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/VYGR/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AFIIQ/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/AXSM/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/APLS/Customers/C 1.csv
/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BGNE/Customers/C 1.cs

/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:
/var/folders/5d/9jrnkbxx5bb_l2v7b5pxxnfm0000gn/T/ipykernel_16119/603308844.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if sum(df['Firm Name'].str.contains(dot_pattern, '...', regex=True))>0:


In [27]:
path

'/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/BSGM/Customers/S 1.csv'

In [24]:
data

,Company,Industry,Mkt Cap (M),Relationship Size (M),% Cost,Cost Cat,Supplier Rev,Source,Size Source
0,VWR Corp,Medical Equipment & Devices,--,--,--,--,--,--,--
1,FirstEnergy Corp,Electric Utilities,"21,982.62",--,--,--,--,Supplier's 2021A CF,--


In [226]:
np.where(tickers=="DSKE")

(array([], dtype=int64),)

In [205]:
df

,ticker,Firm Name,Supplier Name,Industry,Mkt Cap (M),Relationship Size (M),% Cost,Cost Cat,Supplier Rev,Source
0,ACB,Aurora Cannabis Inc,Strainprint Technologies Ltd,--,--,--,--,--,--,NaN
1,ACB,Aurora Cannabis Inc,PAX Labs Inc,--,--,--,--,--,--,NaN
2,ACB,Aurora Cannabis Inc,EnWave Corp,Machinery,40.90,--,--,--,--,2020
3,ACB,Aurora Cannabis Inc,CTT Pharmaceutical Holdings Inc,Biotech & Pharma,0.28,--,--,--,--,NaN
4,ACB,Aurora Cannabis Inc,Xcel Energy Inc,Electric Utilities,"44,883.36",--,--,--,--,2021
5,ACB,Aurora Cannabis Inc,Valens Co Inc/The,Tobacco & Cannabis,--,--,--,--,--,NaN
6,ACB,Aurora Cannabis Inc,Fortis Inc/Canada,Electric Utilities,"27,221.91",--,--,--,--,2021
7,ACB,Aurora Cannabis Inc,American Software Inc/GA,Software,471.78,--,--,--,--,NaN
8,ACB,Aurora Cannabis Inc,WEC Energy Group Inc,Electric Utilities,"36,732.28",--,--,--,--,2021


## Convert to Dict